In [4]:
"""    
    Indicador: Porcentaje de Hectáreas actualizadas con respecto al total de hectáreas que deben actualizarse para cumplir un Hito contractual.
"""

'    \n    Indicador: Porcentaje de Hectáreas actualizadas con respecto al total de hectáreas que deben actualizarse para cumplir un Hito contractual.\n'

<h3>Importación Librerías</h3>

In [5]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_dataframe_terrenos
import funcion_SQL_a_DataFrame

<h3>Parametrización de Variables</h3>

In [6]:
consulta_tRadicados = """ 
select t0.*
	,radicados."Proceso"
	,radicados.vr_nombre_proceso
	,radicados.estado_tramite
	,radicados.actividad
	,radicados.vr_estado_paso
	,radicados.proceso_estado
	,radicados.fecha_radicado
	,radicados.usuario
	,radicados.vr_proces_des
	,radicados.vr_consec_nom
	,radicados.vr_consec_des
	,radicados.vr_consec_tip
	,radicados.vr_step_est
	,radicados.vr_task_nom
	,radicados.vr_task_tip
	,radicados.vr_task_con
	,radicados.vr_task_est
from bpmcat.vw_trmactivos_rad_npn t0
inner join (
	select t1.municipio
	    ,t1.npn
	    ,t1.nro_ficha
	    ,t1.radicado
	    ,(case when t1.vr_nombre_proceso like 'Comple%%' then 'Actualización'
	        when t1.vr_nombre_proceso like 'Forma%%' then 'Actualización'
	        when t1.vr_nombre_proceso like 'Muta%%' then 'Conservación'				
	        when t1.vr_nombre_proceso like 'Recti%%' then 'Conservación'
	        else t1.vr_nombre_proceso
	        end) "Proceso"
	    ,t1.vr_nombre_proceso	
	    ,t1.estado_tramite
	    ,t1.actividad
	    ,t1.vr_estado_paso
	    ,t2.vr_proegn_est proceso_estado
	    ,t2.nb_step_idn
	    ,cast(t2.dt_adt_fchupd as date) dt_adt_fchupd
	    ,t2.vr_proegn_ori
	    ,cast(t1.fecha_radicado as date) fecha_radicado
	    ,t1.usuario
	    ,cast(t2.dt_adt_fchupd as date) fecha_actualizacion_tramite
	    ,t3.vr_proces_des
	    ,t6.vr_consec_nom
	    ,t6.vr_consec_des
	    ,t6.vr_consec_tip
	    ,t4.vr_step_est
	    --,t4.nb_task_idn
	    ,t5.vr_task_nom 
	    ,t5.vr_task_tip
	    ,t5.vr_task_con
	    ,t5.vr_task_est 
	    ,t7.vr_functy_des
	from bpmcat.vw_rpt_tramites_en_proceso t1
	left join bpmcat.tb_bpm_egn_proegn t2 on t1.radicado = t2.vr_proegn_rad
	left join bpmcat.tb_bpm_mdl_proces t3 on t2.nb_proces_idn = t3.nb_proces_idn 
	left join bpmcat.tb_fw_sec_consec t6 on t3.nb_consec_idn = t6.nb_consec_idn 
	left join bpmcat.tb_bpm_egn_step t4 on t2.nb_step_idn = t4.nb_step_idn
	left join bpmcat.tb_bpm_mdl_task t5 on t4.nb_task_idn = t5.nb_task_idn
	left join bpmcat.tb_fw_gui_functy t7 on t5.nb_functy_idn = t7.nb_functy_idn
	) radicados 
on radicados.radicado = t0.radicado;
"""	

columnasRadicadoActualizacionCampo = ['municipio','municipio_estandarizado','zona','npn','radicado','Proceso','estado_tramite','actividad','usuario','vr_proces_des']

columnas_espacializacion_radicadosActualizacionConNPNAtendiendoCampo = ['CODIGO',
	'CODIGO_ANTERIOR',                                                                        
	'municipio',
	'municipio_estandarizado',
	'zona',
    'Proceso',
	'radicado',
	'estado_tramite',
	'actividad',
	'usuario',
	'vr_proces_des',
	'Area_ha_cmt12',
	'SHAPE']

L_MUNICIPIO = ['MariaLaBaja', 'Repelon', 'Baranoa', 'Sabanagrande']
NOMBRE_PROCESOS = ['Actualización','Conservación']
MUNICIPIO_FUENTE = ['MARIA LA ']

DIRECTORIO_REPORTE = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\zReportes"
NOMBRE_REPORTE_RADSINNPN = 'Reporte_Radicado_SinNPNoFicha.xlsx'
NOMBRE_REPORTE_RADFORMACIONATENDIENDO = 'Reporte_Radicado_EnAtencion.xlsx'

RUTA_BD_LOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\Base_Datos\_2_2_0.gdb"
RUTA_BD_CONSOLIDADA = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
NOMBRE_CAPA_HAXESTADO = 'TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS'

XLSX_UNIFICADO = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\zReportes\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas.xlsx"

RUTA_RAIZ = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\zReportes"
NOMBRE_IND_2_2_0 = 'Seguimiento_Diario'

fecha_actual = datetime.now()
fecha_directorio = str(fecha_actual.strftime("%Y-%m-%d"))

<h3>Ejecución de Funciones</h3>

In [7]:
df_terrenos = funcion_dataframe_terrenos.dataframe_terrenos()
df_consulta_radicados = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRadicados)

Concluye el proceso
Inicio de la conexión
Conexión exitosa ...


<h3>Generación de Campo Zona (Urbano/Rural) y Estandarización Campo Municipio</h3>

In [8]:

# TODO: Generacion Campo Zona
df_predios_radicados = df_consulta_radicados.copy()

df_predios_radicados['codigo_zona'] = df_predios_radicados['npn'].str[5:7]
df_predios_radicados['zona'] = None

df_predios_radicados.loc[df_predios_radicados['codigo_zona'] == '01', 'zona'] = 'Urbano'
df_predios_radicados.loc[df_predios_radicados['codigo_zona'] == '00', 'zona'] = 'Rural'
df_predios_radicados.loc[(df_predios_radicados['codigo_zona'] != '00') & (df_predios_radicados['codigo_zona'] != '01'), 'zona'] = 'Corregimientos Urbanos'

# TODO: Generacion Campo municipio_estandarizado
df_predios_radicados['municipio_estandarizado'] = None
df_predios_radicados.loc[df_predios_radicados['municipio'] == 'MARÍA LA BAJA', 'municipio_estandarizado'] = 'MariaLaBaja'
df_predios_radicados.loc[df_predios_radicados['municipio'] == 'REPELÓN', 'municipio_estandarizado'] = 'Repelon'
df_predios_radicados.loc[df_predios_radicados['municipio'] == 'BARANOA', 'municipio_estandarizado'] = 'Baranoa'
df_predios_radicados.loc[df_predios_radicados['municipio'] == 'SABANAGRANDE', 'municipio_estandarizado'] = 'Sabanagrande'

<h3>Trámites Sin NPN Registrado</h3>

In [9]:

# TODO: Radicados sin NPN

# ** 1. Filtro
df_radicados_sin_npn = df_predios_radicados[(df_predios_radicados['npn'].isnull()) | (df_predios_radicados['npn'] == '')] 

total_radicados_SinNPN = df_radicados_sin_npn.shape[0]
print(f" 1. El total de radicados sin parametrización de NPN o Ficha es: \n {total_radicados_SinNPN}")

df_radicadosSinNPNParaReporte = df_radicados_sin_npn[['municipio','Proceso','radicado','usuario','vr_consec_nom']]
df_radicadosSinNPNParaReporte.to_excel(os.path.join(DIRECTORIO_REPORTE, NOMBRE_REPORTE_RADSINNPN), sheet_name='ReporteRadSinNPN')
print(f"Se crea el reporte {NOMBRE_REPORTE_RADSINNPN}")

df_radicadosActualizacionSinNPN = pd.DataFrame([[total_radicados_SinNPN]], columns=['TotalRadicadosSinNPN'])

 1. El total de radicados sin parametrización de NPN o Ficha es: 
 0
Se crea el reporte Reporte_Radicado_SinNPNoFicha.xlsx


<h3>Total Radicados por Zona y Proceso</h3>

In [10]:

df_predios_radicados_x_proceso_x_zona = pd.DataFrame(df_predios_radicados[['municipio','Proceso','zona']].value_counts()).reset_index().rename(columns={'count':'total_predios'})

dataframes = {}

for municipio in L_MUNICIPIO:
    query_municipio_zona = "municipio_estandarizado == "+"'"+municipio+"'"
    df = df_predios_radicados.query(query_municipio_zona)
    dataframes[municipio] = df

for municipio, df in dataframes.items():
    globals()[f'df_{municipio}'] = df
    globals()[f'df_{municipio}_Filtrado'] = pd.DataFrame(globals()[f'df_{municipio}'][['Proceso', 'zona']].value_counts().reset_index()).rename(columns={'count':'total_predios'})
    globals()[f'df_{municipio}_Filtrado_Pivote'] = globals()[f'df_{municipio}_Filtrado'].pivot(index="zona", columns="Proceso", values="total_predios")
    for columna in NOMBRE_PROCESOS:
        if columna in globals()[f'df_{municipio}_Filtrado_Pivote'].columns:
            globals()[f'df_{municipio}_Filtrado_Pivote'].loc[globals()[f'df_{municipio}_Filtrado_Pivote'][columna].isnull(), columna] = 0
            globals()[f'df_{municipio}_Filtrado_Pivote'][columna] = globals()[f'df_{municipio}_Filtrado_Pivote'][columna].astype(int)

# ! Para gráfica
# print(f"Radicados por Zona, Maria La Baja: \n {df_MariaLaBaja_Filtrado}")
# print(f"Radicados por Zona, Repelon: \n {df_Repelon_Filtrado}")
# print(f"Radicados por Zona, Baranoa: \n {df_Baranoa_Filtrado}")
# print(f"Radicados por Zona, Sabanagrande: \n {df_Sabanagrande_Filtrado}")

# # ! Para Tabla
print(F"PARA TABLAS \n")
print(f"Radicados por Zona, Maria La Baja: \n {df_MariaLaBaja_Filtrado_Pivote}")
print(f"Radicados por Zona, Repelon: \n {df_Repelon_Filtrado_Pivote}")
print(f"Radicados por Zona, Baranoa: \n {df_Baranoa_Filtrado_Pivote}")
print(f"Radicados por Zona, Sabanagrande: \n {df_Sabanagrande_Filtrado_Pivote}")

PARA TABLAS 

Radicados por Zona, Maria La Baja: 
 Proceso                 Actualización  Conservación
zona                                               
Corregimientos Urbanos            229             1
Rural                             663             0
Urbano                            605             2
Radicados por Zona, Repelon: 
 Proceso                 Actualización  Conservación
zona                                               
Corregimientos Urbanos            844             0
Rural                             270             1
Urbano                              0             2
Radicados por Zona, Baranoa: 
 Proceso  Conservación
zona                 
Urbano             14
Radicados por Zona, Sabanagrande: 
 Proceso  Conservación
zona                 
Urbano              1


<h3>Predios Atendiendo</h3>

In [11]:
df_radicadosConNPNAtendiendo = df_predios_radicados[(df_predios_radicados['npn'].notnull()) & 
    (df_predios_radicados['vr_estado_paso'] == 'ATENDIENDO')]

df_radicadosConNPNAtendiendoCampo = df_radicadosConNPNAtendiendo[columnasRadicadoActualizacionCampo]
print(f" 1. El total de trámites, en actualización (formación) y que están siendo atendidos son: {df_radicadosConNPNAtendiendo.shape[0]}")

# ! Para Tabla
df_totalprediosActualEnAtencion = pd.DataFrame([[df_radicadosConNPNAtendiendoCampo.shape[0]]], columns=['Total Predios Actualizacion En Atención'])

df_radicadosConNPNAtendiendoCampo.to_excel(os.path.join(DIRECTORIO_REPORTE, NOMBRE_REPORTE_RADFORMACIONATENDIENDO), sheet_name='ReportePrediosEnAtencion')
print(f"Se crea el reporte {NOMBRE_REPORTE_RADFORMACIONATENDIENDO}")

 1. El total de trámites, en actualización (formación) y que están siendo atendidos son: 2632
Se crea el reporte Reporte_Radicado_EnAtencion.xlsx


<h3>Proceso: Atendiendo (ESPACIALIZACIÓN)</h3>

In [12]:

df_terrenos_radicadosConNPNAtendiendoCampo = pd.merge(left=df_terrenos, right=df_radicadosConNPNAtendiendo, left_on="CODIGO", right_on="npn", how="inner")

# # TODO: ExportacionTramitesEnAtencionXEstado
# ** Exportación a BD Consolidada
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{NOMBRE_CAPA_HAXESTADO.upper()}"
ruta_salida_terrenos_ha_x_estado = os.path.join(RUTA_BD_CONSOLIDADA, NOMBRE_CAPA_SALIDA)

df_terrenos_radicadosConNPNAtendiendoCampo.spatial.to_featureclass(location=ruta_salida_terrenos_ha_x_estado)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA} a BD Consolidada")

# ** Exportación a BD Seguimiento Indicadores
df_terrenos_radicadosConNPNAtendiendoCampo.spatial.to_featureclass(location=os.path.join(RUTA_BD_LOCAL,NOMBRE_CAPA_HAXESTADO))
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA} a BD Local 2.2.0")

cannot add field: 'fecha_radicado'
Se exporta la capa Analitica_UT_Consolidada\TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS a BD Consolidada
cannot add field: 'fecha_radicado'
Se exporta la capa Analitica_UT_Consolidada\TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS a BD Local 2.2.0


<h3>Calculo de Hectáreas [Atendiendo por Actualización]</h3>

In [13]:

# TODO: Actualización
# ** Mejora de la tabla
df_radicadosConNPNAtendiendoCampo = df_terrenos_radicadosConNPNAtendiendoCampo[columnas_espacializacion_radicadosActualizacionConNPNAtendiendoCampo]
# ** Filtro de registros asociados a la Actualización
df_radicadosActualizacionConNPNAtendiendoCampo = df_radicadosConNPNAtendiendoCampo[df_radicadosConNPNAtendiendoCampo['Proceso'] == 'Actualización']


In [14]:
df_radicadosActualizacionConNPNAtendiendoCampo_municipio_has = df_radicadosActualizacionConNPNAtendiendoCampo.groupby(['municipio_estandarizado'])['Area_ha_cmt12'].sum()
df_radicadosConNPNAtendiendoCampo_municipio_has = pd.DataFrame(df_radicadosActualizacionConNPNAtendiendoCampo_municipio_has).reset_index().round(2)
df_radicadosActualizacionConNPNAtendiendoCampo_municipio_has

municipio_estandarizado
MariaLaBaja    7200.383745
Repelon         7838.23684
Name: Area_ha_cmt12, dtype: Float64

<h3>Calculo de Hectáreas: Actualizacion - Actualización - Atendiendo - Zona</h3>

In [15]:
df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has = df_radicadosActualizacionConNPNAtendiendoCampo.groupby(['municipio_estandarizado','zona'])['Area_ha_cmt12'].sum()
df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has = pd.DataFrame(df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has).reset_index().round(2)

# ! Para Gráfico
df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has

# ! Para Tabla
df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has_pivote = df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has.pivot(index="zona", columns="municipio_estandarizado", values="Area_ha_cmt12")
df_radicadosConNPNAtendiendoCampo_municipio_zona_has_pivote =  df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has_pivote.fillna(0)
df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has_pivote

municipio_estandarizado,MariaLaBaja,Repelon
zona,,
Corregimientos Urbanos,31.07,32.54
Rural,7149.0,7805.7
Urbano,20.31,<NA>


In [16]:

# ** Usar ExcelWriter para guardar múltiples dataframes en diferentes hojas
with pd.ExcelWriter(XLSX_UNIFICADO, engine='xlsxwriter') as writer:
    df_MariaLaBaja_Filtrado_Pivote.to_excel(writer, sheet_name='5_1_PredActXZonaMariaLaBaja', index=False)
    df_Repelon_Filtrado_Pivote.to_excel(writer, sheet_name='5_2_PredActXZonaRepelon', index=False)
    df_Baranoa_Filtrado_Pivote.to_excel(writer, sheet_name='5_3_PredActXZonaBaranoa', index=False)
    df_Sabanagrande_Filtrado_Pivote.to_excel(writer, sheet_name='5_4_PredActXZonaSabanag', index=False)
    df_radicadosActualizacionConNPNAtendiendoCampo_municipio_has.to_excel(writer, sheet_name='16_TotalHAActuaXMpio', index=False)
    df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has_pivote.to_excel(writer, sheet_name='17_TotalHAActuaXMpioXZona', index=False)
    df_totalprediosActualEnAtencion.to_excel(writer, sheet_name='4_TotalPrediosEnAtencion', index=False)

print(f"Se genera XLSX Unificado {XLSX_UNIFICADO}")

Se genera XLSX Unificado C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\zReportes\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas.xlsx


In [17]:

dir_directorio_salidas_xlsx = {1:[df_MariaLaBaja_Filtrado_Pivote,'5_1_PredActXZonaMariaLaBaja.xlsx','5_1_PredActXZonaMariaLaBaja'],
    2:[df_Repelon_Filtrado_Pivote,'5_2_PredActXZonaRepelon.xlsx','5_2_PredActXZonaRepelon'],
    3:[df_Baranoa_Filtrado_Pivote,'5_3_PredActXZonaBaranoa.xlsx','5_3_PredActXZonaBaranoa'],
    4:[df_Sabanagrande_Filtrado_Pivote,'5_4_PredActXZonaSabanag.xlsx','5_4_PredActXZonaSabanag'],
    5:[df_radicadosActualizacionConNPNAtendiendoCampo_municipio_has,'16_TotalHAActuaXMpio.xlsx','16_TotalHAActuaXMpio'],
    6:[df_radicadosActualizacionConNPNAtendiendoCampo_municipio_zona_has_pivote,'17_TotalHAActuaXMpioXZona.xlsx','17_TotalHAActuaXMpioXZona'],
    7:[df_totalprediosActualEnAtencion,'4_TotalPrediosEnAtencion.xlsx','4_TotalPrediosEnAtencion'],
    }

if os.path.exists(os.path.join(RUTA_RAIZ, NOMBRE_IND_2_2_0, fecha_directorio)):
    for llave, valor in dir_directorio_salidas_xlsx.items():
        valor[0].to_excel(os.path.join(RUTA_RAIZ, NOMBRE_IND_2_2_0, fecha_directorio, valor[1]), sheet_name=valor[2])
        print(f"Se genera XLSX de DF {valor[1]}")
else:
    os.mkdir(os.path.join(RUTA_RAIZ, NOMBRE_IND_2_2_0, fecha_directorio))
    for llave, valor in dir_directorio_salidas_xlsx.items():
        valor[0].to_excel(os.path.join(RUTA_RAIZ, NOMBRE_IND_2_2_0, fecha_directorio, valor[1]), sheet_name=valor[2])
        print(f"Se genera XLSX de DF {valor[1]}")

Se genera XLSX de DF 5_1_PredActXZonaMariaLaBaja.xlsx
Se genera XLSX de DF 5_2_PredActXZonaRepelon.xlsx
Se genera XLSX de DF 5_3_PredActXZonaBaranoa.xlsx
Se genera XLSX de DF 5_4_PredActXZonaSabanag.xlsx
Se genera XLSX de DF 16_TotalHAActuaXMpio.xlsx
Se genera XLSX de DF 17_TotalHAActuaXMpioXZona.xlsx
Se genera XLSX de DF 4_TotalPrediosEnAtencion.xlsx
